<a href="https://colab.research.google.com/github/hosam00767/Arabic-English-Translator-using-transformers/blob/main/Transformer_from_scratch_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
import math

In [16]:
class InputEmbedding(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size,d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)

In [17]:
class PositionalEncoding(nn.Module):

  def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    pe = torch.zeros(seq_len, d_model)
    #creating the positional Embedding vector
    position = torch.zeros(seq_len, d_model).unsqueeze(1)
    div_term = torch.exp(torch.arange(0,d_model,2)*-(math.log(1000.0/d_model)))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer('pe',pe)

  def forward(self, x):
    x = x + self.pe[:, :x.shape[1], :].require_grad_(False)
    return self.dropout(x)



In [18]:
class LayerNormalization(nn.Module):
  def __init__(self, eps: float = 10**-6):
    super().__init__()
    self.eps = eps
    self.bias = nn.Parameter(torch.ones(0)) # Parameter makes the parameter learnable
    self.alpha = nn.Parameter(torch.ones(1))

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim=True)
    std = x.std(dim=-1, keepdim=True) #return mean output with the same shape as x
    return self.alpha * (x-mean)/(std+ self.eps)+  self.bias


In [19]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model, d_ff:  int, dropout: float):
    super().__init__()

    # Bias is True by default inside Linear
    self.linear_1 = nn.Linear(d_model, d_ff) # W1 and B1
    self.dropout()
    self.linear_2 = nn.Linear(d_ff, d_model) # W2 and B2

  def forward(self, x):
    # 1- (Batch, seq_len, d_model) --> (Batch, seq_len, d_ff)
    # 2- (Batch, seq_len, d_ff)    --> (Batch, seq_len, d_model)

    return self.linear2(self.dropout(torch.relu(self.linear_1(x))))


In [20]:
# MultiHead Attention
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self, d_model: int, h: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert d_model % h == 0

    self.d_k = d_model % h
    self.w_q = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_o = nn.Linear(d_model, d_model)

    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout):
    d_k = query.shape[-1]

    attention_scores = (query @ key.transpose(-2,-1)) / math.sqrt(d_k)

    if mask is not None:
      attention_scores.masked_fill_(mask == 0, -1e9)
    attention_scores = attention_scores.softmax (dim=-1)

    if dropout is not None:
      attention_scores = dropout(attention_scores)

    return (attention_scores@value), attention_scores

  def forward(self, q, v, k, mask):
    query = self.w_q(q)
    value = self.w_v(v)
    key = self.w_k(k)
    #view function is used to reshape a tensor
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
    key = key.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
    value = value.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)

    x, self.attention_scores = MultiHeadAttentionBlock.attention(q,k,v,mask,self.dropout)

    x = x.transpose(1,2 ).continguous().view(x.shape[0],-1,self.h * self.d_k )
    x = self.w_o(x)
    return x



In [21]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout: float)-> None:
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

In [22]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention_block:MultiHeadAttentionBlock, feed_forward_block:FeedForwardBlock, dropout: float)->None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for i in range(2)])

  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
    x = self.residual_connections[1](x, self.feed_forward_block)


In [23]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x,mask)

In [24]:
class DecoderBlock(nn.Module):
  def __init__(self, cross_attention_block: MultiHeadAttentionBlock, self_attention_block:MultiHeadAttentionBlock, feed_forward_block:FeedForwardBlock, dropout: float)->None:
    super().__init__()

    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tar_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tar_mask))
    x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connections[2](x, self.feed_forward_block)
    return x


In [25]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers

  def forward(self, x, encoder_output, src_mask, tar_mask):
    for layer in self.layers:
      x= layer(x, encoder_output, src_mask, tar_mask)
    return self.norm(x)


In [26]:
class ProjectionLayer(nn.Module):
  def __init__(self, d_model, vocab_size)-> None:
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    return torch.log_softmax(self.proj(x), dim=-1)



In [27]:
class Transformer(nn.Module):
  def __init__(self, encoder: Encoder, decoder: Decoder, src_embedding: InputEmbedding, tar_embedding: InputEmbedding, src_pos: PositionalEncoding, tar_pos: PositionalEncoding, projection_layer: ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embedding = src_embedding
    self.tar_embedding = tar_embedding
    self.src_pos = src_pos
    self.tar_pos = tar_pos
    self.projection_layer = projection_layer


  def encode(self, src, src_mask):
    src = self.src_embedding(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tar, tar_mask):
    tar = self.tar_embedding(tar)
    tar = self.tar_pos(tar)
    tar = self.decoder( tar, encoder_output, src_mask, tar_mask)
    return tar

  def project(self, x):
    return self.projection_layer(x)


In [28]:
def build_Transformer(src_vocab_size: int, tar_vocab_size: int, src_seq_len: int, tar_seq_len: int, d_model=512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048)-> Transformer:
  src_embedding = InputEmbedding(d_model, vocab_size=src_vocab_size)
  tar_embedding = InputEmbedding(d_model, vocab_size=tar_vocab_size)

  # postional encoding
  src_positional_encoding = PositionalEncoding(d_model, src_seq_len, dropout)
  tar_positional_encoding = PositionalEncoding(d_model, tar_seq_len, dropout)

  encoder_blocks= []
  for _ in range(N):
    encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    encoder_feedforward_block = FeedForwardBlock(d_model, d_ff, dropout)
    encoder_block = EncoderBlock(encoder_self_attention_block, encoder_feedforward_block, dropout)
    encoder_blocks.append(encoder_block)

  decoder_blocks= []
  for _ in range(N):
    decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    decoder_feedforward_block = FeedForwardBlock(d_model, d_ff, dropout)
    decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, decoder_feedforward_block, dropout)
    decoder_blocks.append(decoder_block)

  encoder = Encoder(nn.ModuleList(encoder_blocks))
  decoder = Decoder(nn.ModuleList(decoder_blocks))
  projection_layer = ProjectionLayer(d_model= d_model, vocab_size= tar_vocab_size)

  transformer = Transformer(encoder, decoder, src_embedding, tar_embedding, src_positional_encoding, tar_positional_encoding, projection_layer)
  # intialize the parameter
  for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return transformer
